In [1]:
import pandas as pd
import numpy as np
from io import BytesIO
import requests
from urllib.parse import urlencode
import urllib
from requests.auth import HTTPBasicAuth
from requests.exceptions import ChunkedEncodingError
import os
import json
import yadisk
from datetime import datetime, date, timedelta
import locale
from time import sleep
import shutil
import gc
import turbodbc
from turbodbc import connect
import gc
from pandas.api.types import is_string_dtype
import numpy as np
from sqlalchemy import create_engine
import pyodbc
import warnings
import re

import config
from yandex_disk_func import *
# from parse_functions import *



# # указываем путь и название файла с токеном для Яндекс Диск
file_path = r'C:\Users\o.bogomolov\Desktop\Jupyter_notebook\38_x5_report_parser'

public_key = config.public_key # обычная ссылка на доступ к папке одного данного ФЛАЙТА из личного кабинета

# забираем список папок в нужной нам директории
res = get_yandex_disk_folders(public_key)
yandex_folders = res.json() # парсим ответ

200


In [2]:
# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
pd.options.mode.chained_assignment = None

In [3]:
public_key = yandex_folders['public_key']  # из ответа Яндекс забираем public_key, чтобы использовать его для скачивания файлов

for i in range(len(yandex_folders['_embedded']['items'])): # через цикл проходим по ответу Яндекса и забираем названия вложенных папок
    file_type = yandex_folders['_embedded']['items'][i]['type']
    if file_type=='dir':   # если находим файлы с типом dir (папка), то забираем путь к этой папке
        folder_path = yandex_folders['_embedded']['items'][i]['path']
        print(folder_path)
        # yandex_responce = get_yandex_disk_responce(base_public_url, public_key, folder_path) # отправляем запрос, чтобы получить содержимое папки

        # # Через цикл проходим по папке с файлами
        # # Нас интересуют файлы эксель. Причем каждая экселька будет парситься по своему, т.к. они относятся к разным рекламным площадкам
        
        # # Проходим через цикл по содержимому папки (отдельный флайт)
        # for i in range(len(yandex_responce['_embedded']['items'])):
        #     file_info = yandex_responce['_embedded']['items'][i]
        #     if file_info['type']=='file':  # если документ является фалйом(не папкой или изображением), то забираем его название 
        #         file_name = file_info['name'] # сохраняем название файла
        #         if 'xls' in file_name: # еслит тип файла является xlsx, то уберем расширение и будем его использовать в качесвте названия отчета
        #             file_path = file_info['path']
                    
        #             # report_name = '.'.join(file_name.split('.')[:-1]) # убираем .xlsx из названия файла
        #             print(file_name)
        #             res_file_link = get_yandex_disk_responce(download_url, public_key, file_path) # получаем ссылку на скачивание отчета
        #             download_response = requests.get(res_file_link['href'])

/Доставка
/Перекресток
/Пятерочка


In [ ]:
data_link = download_response.content

In [ ]:
sheet_names = pd.ExcelFile(BytesIO(data_link))

In [ ]:
sheet_names.sheet_names

In [ ]:
network = 'test'
report_name = 'tst'
tmp_dict = {}

In [ ]:
for sheet_name in sheet_names.sheet_names:
    if 'mediaplan' in sheet_name:
        df = pd.read_excel(BytesIO(data_link), sheet_name=sheet_name, header=None)
        
        # приводим в порядок название листа, чтобы его записать в новую таблицу
        sheet_name = normalize_headers(sheet_name)
        print(f'    {sheet_name}')
        
        # заполняем вниз название истоника и аудитории
        ffill_columns = [1, 2]
        df[ffill_columns] = df[ffill_columns].ffill() # заполняем вниз
        # заполяем вниз rotation type - здесь объединенная ячейка, и в этом поле нет названия
        # чтобы оно появилось, заранее протягиваем вниз это название
        df[8] = df[8].fillna('rotation type')
        # заполняем вниз объединенные ячейки
        df = df.fillna('')
        
        # сохраняем название бренда
        brand = df[3].loc[get_index_row(df, 2, 'brand')] 
        # сохраняем период
        period = df[3].loc[get_index_row(df, 2, 'период')]
        
    
        # забираем индекс начала таблицы
        start_index = get_index_row(df, 1, 'category')
        # задаем названия полей
        df.columns = df.iloc[start_index].apply(normalize_headers) # забираем название полей из файла
        # обрезаем верхнюю часть таблицы. она больше не нужна
        df = df.iloc[start_index+2:].reset_index(drop=True)
        # забираем окончание таблицы
        end_index = get_index_row(df, 'category', 'total')
        # обрезаем таблицу снизу
        df = df.iloc[:end_index].reset_index(drop=True)

In [ ]:
df

In [ ]:
# источник FirstData
# типы размещения Видео и Баннерная реклама
# Функция для обработки медиаплана 
# 1. Медиаплан для обработки находится на листе Plan_Media 
# 2. В столбике B  находится слово Brand справа от него в столбике C название Бренда
# 3. В столбике B  находится слово Period справа от него в столбике C название указан период медиаплана
# 4. В столбике B  должно находиться поле Source
# 5. Каждая таблица должна заканчиваться строкой итогов 
# 6. В столбике С находятся Таргетинги - Targeting by purchase 
# 7. В столбике I находится тип размещения (CPC, CPM)

def get_firstdata_mediaplan(data_link, network, report_name):
    tmp_dict = {}
    sheet_names = pd.ExcelFile(BytesIO(data_link))
    for sheet_name in sheet_names.sheet_names:
        if 'mediaplan' in sheet_name:
            df = pd.read_excel(BytesIO(data_link), sheet_name=sheet_name)
            # приводим в порядок название листа, чтобы его записать в новую таблицу
            sheet_name = normalize_headers(sheet_name)
            print(f'    {sheet_name}')
            # заполняем вниз название истоника
            df['Unnamed: 1'] = df['Unnamed: 1'].ffill()
            # заполняем вниз таргетинги
            df['Unnamed: 2'] = df['Unnamed: 2'].ffill()
            # заполяем вниз rotation type - здесь объединенная ячейка, и в этом поле нет названия
            # чтобы оно появилось, заранее протягиваем вниз это название
            df['Unnamed: 8'] = df['Unnamed: 8'].fillna('rotation type')
            # заполняем вниз объединенные ячейки
            df = df.fillna('')
            
            # сохраняем название бренда
            brand = df['Unnamed: 3'].loc[get_index_row(df, 'Unnamed: 2', 'brand')] 
            # сохраняем период
            period = df['Unnamed: 3'].loc[get_index_row(df, 'Unnamed: 2', 'период')]
            # забираем индекс начала таблицы
            start_index = get_index_row(df, 'Unnamed: 1', 'category')
            
            # задаем названия полей
            df.columns = df.iloc[start_index].apply(normalize_headers) # забираем название полей из файла
            # обрезаем верхнюю часть таблицы. она больше не нужна
            df = df.iloc[start_index+2:].reset_index(drop=True)
            # забираем окончание таблицы
            end_index = get_index_row(df, 'category', 'total')
            # обрезаем таблицу снизу
            df = df.iloc[:end_index].reset_index(drop=True)
            # создаем базовый список полей, которые есть всегда вне зависимости от типа размещения
            standart_columns = ['category', 'targeting by purchase', 'format', 'period', 
                            'price list cost (cost per unit) net ', 'rotation type',
                            'total price list cost net', 'reach forecast (uu)', 'frequency total till',
                            'impressions', 'clicks']
            # проверяем наличие Видео размещений. Если они есть, то используем дополнительные поля из таблицы
            # если Видео размещений нет, то добавляем дополнительно 2 поля с 0 (это нужно для нормализации общей таблицы     
            if 'vtr,%' not in list(df.columns):
                df['number of views'] = 0
                df['vtr,%'] = 0.0
                
            standart_columns += ['number of views', 'vtr,%']
            # оставляем только нужные поля
            df = df[standart_columns]
            # приводим названия полей к единому стандарту
            df = df.rename(columns={'category': 'source', 'targeting by purchase': 'targeting', 'format': 'ad copy format', 
                                     'price list cost (cost per unit) net ': 'unit price', 
                                    'frequency total till': 'frequency', 'reach forecast (uu)': 'reach', 
                                    'total price list cost net' :'budget_without_nds', 'number of views': 'views', 'vtr,%': 'vtr, %'})
    
            # некоторые типы размещений имеют объединенные строки
            # например Баннер и универсальный баннер - это 2 строки с объединенными ячейками по расходам, показам и тд.
            # поэтому создадим доп. поле, где соединим их названия в одну строку
            # создаем пустое поле
            df['merge_type_cells'] = ''
            # проходим через цикл по датаФрейму
            # в первой строке по определению не может быть данных, поэтому сохраняем там название формата
            # если это не первая и не посленяя строка, то нам нужно провести проверку
            # допустим мы находимся в строке номер 2
            # мы проверяем, что находится в строке номер 3 в поле rotation type (на первых шагах мы сделали заполнение вниз)
            # соответсвенно если ячейка была пустая, там появится надпись rotation type - так мы поймем, что это как раз объединенные данные
            # берем название формата из текущей строки и добавляем к нему название формата из следующей строки
            # во всех остальных случаях просто возвращаем название формата из текущей строки
            for i in range(len(df)):
                base_name = df['ad copy format'][i]
            
                if i < len(df)-1:
                    if df['rotation type'][i+1] == 'rotation type':
                        base_name = base_name + ' / ' + str(df['ad copy format'][i+1])
                else:
                    base_name = df['ad copy format'][i]
                df['merge_type_cells'][i] =  base_name
    
            # последняя строка является объединенной Сначала идет строка Баннеры - в ней все цифры
            # Второая строка Универсальные баннеры - в ней нет значений потому что поставщик считает, что это одно и тоже
            # мы убираем такие строки без данных
            df = df[df['impressions']!='']
            # передаем новое название формата в нужное нам поле
            df['ad copy format'] = df['merge_type_cells']
            df = df.drop('merge_type_cells', axis=1)
            # добавляем поля с общей информацией
            df['supplier'] = network
            df['report_name'] = report_name
            df['sheet_name'] = sheet_name
            df['brand'] = brand
            df['period'] = period
            df['site/ssp'] = ''
            df['placement'] = ''
            df['budget_nds'] =(df['budget_without_nds'] * 1.2).astype('float').round(2)
           
            # парсим Гео и Соц. дем из поля targeting
            df['geo'] = df['targeting'].apply(lambda x: get_targetings(x, 'гео:','\n', flag='geo'))
            df['soc_dem'] = df['targeting'].apply(lambda x: get_targetings(x, 'ца:','покупатели', flag='soc'))
            
            # если в этих полях встречаются пустые ячейки, то заменяем их на 0
            df['vtr, %'] = df['vtr, %'].apply(replace_blank)
            df['views'] = df['views'].apply(replace_blank)
            # переставляем поля местами, чтобы все было единообразно
            df = df[base_cols]
            tmp_dict[sheet_name] = df

    return pd.concat(tmp_dict, ignore_index=True)

In [ ]:
network = 'weborama'
report_name = 'tst'
tmp_dict = {}
sheet_names = pd.ExcelFile(BytesIO(data_link))
# в этом файле присутствуют скрытые листы
# нам нужно исключить их из парсинга
# поэтому добавляем дополнительный блок с проверкой статуса листа
sheets = get_sheets_list(sheet_names, extention)
for sheet in sheets:
    sheet_name = check_excel_sheets(sheet, extention)
    if sheet_name:
        if 'banner' in sheet_name.lower() or 'video' in sheet_name.lower() \
        or 'баннер' in sheet_name.lower() or 'видео' in sheet_name.lower():
            
            df = pd.read_excel(BytesIO(data_link), sheet_name=sheet_name, header=None)
            sheet_name = normalize_headers(sheet_name)
            print(sheet_name)
            # заголовки в файле состоят из 2-х строк, поэтому нужно выполнить заполнение вниз на 1 строку
            # забираем название полей, в которых нужно сдвинуть строку вниз
            ffill_columns = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
            df[ffill_columns] = df[ffill_columns].ffill(limit=1) # заполняем вниз
            
            # т.к. мы выполнили заполнение вниз на 1 строку
            # у нас дублируются заголовки, поэтому мы берем второе вхождение забираем индекс начала таблицы
            start_index = get_index_row(df, 1, 'период') + 1
            # забираем название полей из файла
            col_names_list = df.iloc[start_index].fillna('').apply(normalize_headers)
            pattern = 'показы'
            # получаем индекс колонки
            col_index = get_col_index(col_names_list, pattern, flag='equals')
            df[col_index] = df[col_index].fillna('0')
            df = df.fillna('')

            # сохраняем название бренда
            brand = df[1].loc[get_index_row(df, 1, 'клиент')]
            brand = brand[len('Клиент:'):].strip()
            
            # если целевая аудитория НЕ будет указана в самой таблице
            # то возьмем значение из описания НАД таблицей
            ca = df[1].loc[get_index_row(df, 1, 'ца')]
            ca = ca[len('ЦА:'):].strip()
            
                        
            # т.к. одно из полей может менять название, нам нужно его к единому стандарту
            # сначала получим индекс колонки, которая нам нужна. она тоже может менять положение (может быть втоорой или третьей)
            pattern = 'сервис'
            # получаем индекс колонки
            col_index = get_col_index(col_names_list, pattern)
            # получаем индекс строки
            # row_index = get_index_row(df, col_index, pattern) + 1
            # меняем название в нужнй ячейке
            df[col_index].loc[start_index] = 'source'
            
            # меняем назвние еще одного поля
            pattern = 'объем'
             # получаем индекс колонки
            col_index = get_col_index(col_names_list, pattern)
            # получаем индекс строки
            # row_index = get_index_row(df, col_index, pattern) + 1
            # меняем название в нужнй ячейке
            df[col_index].loc[start_index] = 'budget_without_nds'

            # меняем назвние еще одного поля
            pattern = 'частота'
             # получаем индекс колонки
            col_index = get_col_index(col_names_list, pattern)
            # получаем индекс строки
            # row_index = get_index_row(df, col_index, pattern) + 1
            # меняем название в нужнй ячейке
            df[col_index].loc[start_index] = 'frequency'
            
            # задаем названия полей
            df.columns = df.iloc[start_index].apply(normalize_headers) # забираем название полей из файла
            # обрезаем верхнюю часть таблицы. она больше не нужна
            df = df.iloc[start_index+1:].reset_index(drop=True)
            # обрезаем таблицу снизу
            end_index = get_index_row(df, 'показы', '0')
            df = df.iloc[:end_index]
            # создаем базовый список полей, которые есть всегда вне зависимости от типа размещения
            standart_columns = ['период', 'source', 'формат рекламы', 'гео', 'аудиторные данные перечислены через слэш (/)',
                            'стоимость за единицу (до ндс)', 'budget_without_nds', 'показы', 'охват', 'клики', 'frequency', 'модель закупки']
            
            # проверяем наличие Видео размещений. Если они есть, то используем дополнительные поля из таблицы
            # если Видео размещений нет, то добавляем дополнительно 2 поля с 0 (это нужно для нормализации общей таблицы     
            if 'vtr' not in list(df.columns):
                df['views'] = 0
                df['vtr'] = 0.0
            else:
                df['views'] = df['vtr'].astype('float') * df['показы'].astype('float')
                
            if 'ца' not in list(df.columns):
                df['ца'] = ca

            standart_columns += ['ца', 'views', 'vtr']
            # оставляем только нужные поля
            df = df[standart_columns]
            # приводим названия полей к единому стандарту
            df = df.rename(columns={'период': 'period', 'формат рекламы': 'ad copy format', 'гео': 'geo',
                                    'аудиторные данные перечислены через слэш (/)': 'targeting', 'ца': 'soc_dem', 
                                    'модель закупки': 'rotation type', 'стоимость за единицу (до ндс)': 'unit price',
                                    'показы': 'impressions', 'охват': 'reach', 'клики': 'clicks', 'vtr': 'vtr, %'})
            df['supplier'] = network
            df['report_name'] = report_name
            df['sheet_name'] = sheet_name
            df['brand'] = brand
            df['site/ssp'] = ''
            df['placement'] = ''
            df['budget_nds'] =(df['budget_without_nds'] * 1.2).astype('float').round(2)
            
            df = df[base_cols]

In [ ]:
df

In [ ]:
public_key = yandex_folders['public_key']  # из ответа Яндекс забираем public_key, чтобы использовать его для скачивания файлов

for i in range(len(yandex_folders['_embedded']['items'])): # через цикл проходим по ответу Яндекса и забираем названия вложенных папок
    file_type = yandex_folders['_embedded']['items'][i]['type']
    if file_type=='dir':   # если находим файлы с типом dir (папка), то забираем путь к этой папке
        folder_path = yandex_folders['_embedded']['items'][i]['path']
        print(folder_path)
        yandex_responce = get_yandex_disk_responce(base_public_url, public_key, folder_path) # отправляем запрос, чтобы получить содержимое папки

        # Через цикл проходим по папке с файлами
        # Нас интересуют файлы эксель. Причем каждая экселька будет парситься по своему, т.к. они относятся к разным рекламным площадкам
        
        # Проходим через цикл по содержимому папки (отдельный флайт)
        for i in range(len(yandex_responce['_embedded']['items'])):
            file_info = yandex_responce['_embedded']['items'][i]
            if file_info['type']=='file':  # если документ является фалйом(не папкой или изображением), то забираем его название 
                file_name = file_info['name'] # сохраняем название файла
                if 'pdf' in file_name: # еслит тип файла является xlsx, то уберем расширение и будем его использовать в качесвте названия отчета
                    file_path = file_info['path']
                    
                    # report_name = '.'.join(file_name.split('.')[:-1]) # убираем .xlsx из названия файла
                    print(file_name)
                    res_file_link = get_yandex_disk_responce(download_url, public_key, file_path) # получаем ссылку на скачивание отчета
                    download_response = requests.get(res_file_link['href'])


In [ ]:
from functions_parse_pdf import *
from pypdf import PdfReader
import re

In [ ]:
data_link = download_response.content

In [ ]:
test = parse_pdf_benchmarks(data_link)

In [ ]:
test

In [ ]:
def parse_pdf_benchmarks(data_link):
    main_dict = {}
    # открываем PDF документ
    reader = PdfReader(BytesIO(data_link))
    # получаем кол-во страниц в документе
    # проходим через цикл по всем страницам
    print(len(reader.pages))
    for num in range(len(reader.pages)):
        # берем отдельную страницу 
        page = reader.pages[num]
        # забираем из нее содержание
        text = page.extract_text()
        # пробуем выполнить парсинг данных
        # - если получится, то вернем датаФрейм
        # - иначе пропускаем страницу, на которой возникает ошибка парсинга
        try:
            # забираем заголовок таблицы на странице (здесь указывается тип рекламы и категории - VK Реклама в категории авто)
            report_name_index = text.find('\n')
            report_name = text[:report_name_index].lower()
            # забираем период в отношении, которого действуют бенчмарки
            period_row_index = text.find('\n', text.find('\n')+1)
            period_row = text[report_name_index+1: period_row_index]
            first_digit_period = re.search('(\d+)', period_row).group()
            first_digit_index = period_row.find(first_digit_period)
            period = period_row[first_digit_index:]
            
        except:
            pass
        # если в заголовке страницы содержится инфо о отм, что на странице присутствует инфо о бенчмарках, то продолжаем парсинг
        if 'бенчмарки' in report_name and ('vk' in report_name or 'mytarget' in report_name or 'вконтакте' in report_name):
            if 'vk' in report_name:
                print(report_name)
                source = 'VK Ads'
                df_tmp = get_parse_vk_benchmarks(text, report_name, period, source)
                main_dict[report_name] = df_tmp
            elif 'mytarget' in report_name or 'вконтакте':
                print(report_name)
                if 'mytarget' in report_name:
                    source = 'MyTarget'
                else:
                    source = 'Vkontakte'
                df_tmp = get_parse_mytarget_benchmarks(text, report_name, period, source)
                main_dict[report_name] = df_tmp

    return pd.concat(main_dict, ignore_index=True)